In [298]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import sklearn
import pandas.core.algorithms as algos
import math
import traceback
import re
from pandas import Series
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot, plot 
from plotly import tools 
import plotly.graph_objs as go 
import operator
init_notebook_mode(connected=True)
def plotly_df(df, title = ''): 
    data = [] 
    for column in df.columns: 
        trace = go.Scatter( 
            x = df.index, 
            y = df[column], 
            mode = 'lines', 
            name = column 
        ) 
        data.append(trace) 
        
    layout = dict(title = title, showlegend = True) 
    fig = dict(data = data, layout = layout) 
    iplot(fig, show_link=False)
setattr(pd.DataFrame, 'plotly', plotly_df)

def plotly_bar_df(df, title = ''): 
    data = [] 
    for column in df.columns: 
        trace = go.Bar( 
            x = df.index, 
            y = df[column],
            name = column
        ) 
        data.append(trace) 
        
    layout = dict(title = title, showlegend = True) 
    fig = dict(data = data, layout = layout) 
    iplot(fig, show_link=False)
setattr(pd.DataFrame, 'plotly_bar', plotly_bar_df)


import requests
from io import StringIO
def click_to_df(sql, http='http://172.27.42.47:8123'):
    r = requests.post(http, data=(sql + ' FORMAT CSVWithNames'))
    if r:
        return pd.read_csv(StringIO(r.text))
    return print(r.text)
from pyhive import hive
con = hive.Connection(host = 'youlahdp-main6.p', port = 10000, username = 'm.daminova')
curs = con.cursor()

def sql_to_df(sql, conn, buffer = 1000000):
    cursor = conn.cursor()
    cursor.execute(sql)
    clmns = [r[0] for r in cursor.description]
    dfs = list()
    f = cursor.fetchone()
    while f:
        res = []
        i = 0
        while f and i < buffer:
            res.append(f)
            i += 1
            f = cursor.fetchone()
        dfs.append(pd.DataFrame(res, columns = clmns))
    return pd.concat(dfs)
from datetime import timedelta
from datetime import datetime as dates
def daterange(start_date, end_date,tm=1):
    delta = timedelta(days=-tm)
    while  end_date >= start_date:
        yield end_date
        end_date += delta
        
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
    
from hdfs import InsecureClient
client_hdfs = InsecureClient('http://youlahdp-main5.p:50070', user='m.daminova')
    
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)  

import plotly.express as px
from scipy.stats import mannwhitneyu
import scipy.stats as ss
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import statsmodels.formula.api as sfa
import scikit_posthocs as sp
from statsmodels.stats.power import TTestIndPower

curs.execute("set hive.tez.container.size=16000")
curs.execute("set tez.resource.memory.mb=16000")
curs.execute("set mapreduce.job.reduces=200")

## Контакты: предыдущий экран из миниапп

In [88]:
df_contact_total = sql_to_df("""
select count(distinct coalesce( sender_id,client_user_id)) users,
     count(distinct event_id) events,
     count(distinct product_id) prods,
    get_json_object(sources,'$.previous.screen') screen,
    dt
from topics.y_user_user_counter
where dt>='2022-02-10' and dt<='2022-03-03'
  and event_name in ('usr.presscall','chat.first_message')
  and coalesce(client_platform,platform) = 'vk_mini_app'
group by  dt,
    get_json_object(sources,'$.previous.screen')
   """,con)

In [89]:
df_contact_total['screen_new'] = ['None' if cell is None else cell for cell in df_contact_total['screen']]

In [90]:
df_contact_total = df_contact_total.sort_values(by=['dt','screen_new'])

In [91]:
name_screen = {
"None":"Неизвестные и прямые переходы",
"personal_search": 'Сохраненные поиски',
"vk_recommendations" : "Портлеты",
"carousel_preview": "Карусель на главной ВКО превью",
"product_active" : "Профиль продавца",
"carousel_vk_group_manual" : "Портлеты (антибарахолки)",
"main":"Главная раздела" ,
"similars":"Похожие",
"subscribe_similar":"Подписка на похожие",
"seller_subscribe":"Подписка на продавца",
"search":"Поиск в миниаппе",
"lent_from_widget":"Антибарахолки",
"widget_from_post":"Антибарахолки",
"carousel_feed":"Карусель на главной ВКО",
"search_classifieds":"Поиск в ВК",
"snippet":"Сниппет",
"favourites":"Закладки",
"product":"Карточка товара",
"push":"Уведомление",
"vk_lent_advertisement":"Рекламные ссылке в ленте ВК" }

In [92]:
df_contact_total['screen_new_alias'] = df_contact_total['screen_new'].map(name_screen) 

In [295]:
df_contact_total

,users,events,prods,screen,dt,screen_new,screen_new_alias
100,2449,3420,3069,None,2022-02-10,None,Неизвестные и прямые переходы
230,60,80,72,carousel_feed,2022-02-10,carousel_feed,Карусель на главной ВКО
333,403,514,477,carousel_preview,2022-02-10,carousel_preview,Карусель на главной ВКО превью
126,3,3,3,carousel_vk_group_manual,2022-02-10,carousel_vk_group_manual,Портлеты (антибарахолки)
285,659,964,836,favourites,2022-02-10,favourites,Закладки
176,85,96,90,lent_from_widget,2022-02-10,lent_from_widget,Антибарахолки
105,4836,7463,6385,main,2022-02-10,main,Главная раздела
343,276,412,365,personal_search,2022-02-10,personal_search,Сохраненные поиски
28,56,62,59,product,2022-02-10,product,Карточка товара
24,2829,3073,2792,product_active,2022-02-10,product_active,Профиль продавца


In [94]:
fig = px.line(df_contact_total[df_contact_total['dt']>='2022-02-11'], x="dt", y="events",color='screen_new_alias',title='Распределение контактов, события')
fig.show()

In [125]:
df_contact_gr = df_contact_total[df_contact_total.dt>='2022-02-11'].groupby(['dt', 'screen_new_alias']).agg({'events': 'sum'})
totals = df_contact_total[df_contact_total.dt>='2022-02-11'].groupby(['dt']).agg({'events': 'sum'})
total_contact_perc = df_contact_gr.div(totals, level='dt') * 100
total_contact_perc = total_contact_perc.reset_index()

In [127]:
fig = px.line(total_contact_perc, x="dt", y="events",color='screen_new_alias',title='Распределение контактов,%')
fig.show()

In [130]:
round(total_contact_perc.groupby('screen_new_alias').agg({'events': 'mean'}).sort_values(by='events',ascending=False),2)

,events
screen_new_alias,
Неизвестные и прямые переходы,24.28
Сниппет,22.53
Поиск в миниаппе,16.01
Профиль продавца,10.82
Портлеты,7.46
Рекламные ссылке в ленте ВК,4.97
Похожие,4.39
Уведомление,3.09
Закладки,1.45


## Просмотры и конверсия в контакт: миниапп

In [ ]:

create table mdaminova.tmp_contacts as
select dt,
coalesce(client_user_id,sender_id) AS  user_id,
product_id,
count(distinct event_id) contact
from topics.y_user_user_counter
where dt>='2022-02-10' and dt<='2022-03-03'
and event_name in ('usr.presscall','chat.first_message')
and coalesce(client_platform,platform) = 'vk_mini_app'
group by   dt,
coalesce(client_user_id,sender_id),
product_id

In [299]:
# --concat(coalesce(search_type,'пустой_поиск'),'_',coalesce(source_view,'пустой_источник'),'_',coalesce(source,'пустой_vk')) 

df_view_total = sql_to_df("""
select v.dt,
    CASE WHEN search_type='vk_recommendations' then 'Портлеты'
        WHEN source= 'wall%' and search_type in ('feed','group') then 'Сниппеты'
        WHEN source= 'widget%' and search_type in ('feed','group','profile') then 'Антибарахолки'
        WHEN search_type in ('lent_from_widget','widget_from_post','widget') then 'Антибарахолки'
        WHEN search_type = 'carousel_vk_group_manual' then 'Антибарахолки портлеты'
        WHEN search_type = 'vk_lent_advertisement' then 'Рекламные ссылки'
        WHEN search_type = 'search_classifieds' then 'Поиск ВК'
        WHEN search_type = 'push' and source = 'notification' then 'Пуш'
        WHEN search_type = 'lent' and source = 'notification' then 'Пуш'
        WHEN search_type = 'favourites' then 'Закладки'
        WHEN source_view in ('personal_search','subscribe_similar','seller_subscribe') then 'Подписки'
        WHEN source_view in ('similar') then 'Похожие товары'
        WHEN source_view in ('seller_profile') then 'Профиль продавца'
        WHEN source_view in ('all_carousel_products_page','carousel_products_block') then 'Карусели'
        WHEN search_type in ('lent') and source_view is Null then 'Лента ВКО'
        WHEN search_type is Null  and source_view is Null and source is null then 'Неизвестно'
        WHEN search_type ='group' and source_view is Null and source is null then 'Группы'
        WHEN search_type ='feed' and source_view is Null and source is null then 'Лента ВК'
        ELSE 'Остальные'
    END source,
    count(distinct v.user_id) v_users,
    count(distinct c.user_id) c_users,
    sum(views) views,
    sum(contact) contact,
    count(distinct v.product_id) v_products,
    count(distinct c.product_id) c_products
from 
    (select dt, 
        user_id,
        product_id, 
        count(distinct event_id) views,
        max(search_type) search_type,
        max(source_view) source_view,
        max(source) source
    from (select dt,
            coalesce(client_user_id,sender_id) AS user_id,
            product_id,
            event_id,
            FIRST_VALUE(search_type) over(partition by dt, coalesce(client_user_id,sender_id),product_id order by event_timestamp) search_type,
            FIRST_VALUE(source_view) over(partition by dt, coalesce(client_user_id,sender_id),product_id order by event_timestamp) source_view,
            FIRST_VALUE(case when vk_source like 'wall%' then 'wall%' 
                    when vk_source like 'widget%' then 'widget%' 
                    else vk_source 
                end) over(partition by dt, coalesce(client_user_id,sender_id),product_id order by event_timestamp) source
        from topics.y_user_product_counter_new
        where dt>='2022-02-10' and dt<='2022-03-03'
          and event_name = 'prod.view'
          and coalesce(client_platform,platform) = 'vk_mini_app') v 
    where user_id<>'None' and user_id is not null and length(user_id)=24
    group by dt, 
          user_id,
          product_id) v
left join  mdaminova.tmp_contacts c
    on v.dt=c.dt and v.user_id=c.user_id and v.product_id=c.product_id
group by v.dt,
    CASE WHEN search_type='vk_recommendations' then 'Портлеты'
        WHEN source= 'wall%' and search_type in ('feed','group') then 'Сниппеты'
        WHEN source= 'widget%' and search_type in ('feed','group','profile') then 'Антибарахолки'
        WHEN search_type in ('lent_from_widget','widget_from_post','widget') then 'Антибарахолки'
        WHEN search_type = 'carousel_vk_group_manual' then 'Антибарахолки портлеты'
        WHEN search_type = 'vk_lent_advertisement' then 'Рекламные ссылки'
        WHEN search_type = 'search_classifieds' then 'Поиск ВК'
        WHEN search_type = 'push' and source = 'notification' then 'Пуш'
        WHEN search_type = 'lent' and source = 'notification' then 'Пуш'
        WHEN search_type = 'favourites' then 'Закладки'
        WHEN source_view in ('personal_search','subscribe_similar','seller_subscribe') then 'Подписки'
        WHEN source_view in ('similar') then 'Похожие товары'
        WHEN source_view in ('seller_profile') then 'Профиль продавца'
        WHEN source_view in ('all_carousel_products_page','carousel_products_block') then 'Карусели'
        WHEN search_type in ('lent') and source_view is Null then 'Лента ВКО'
        WHEN search_type is Null  and source_view is Null and source is null then 'Неизвестно'
        WHEN search_type ='group' and source_view is Null and source is null then 'Группы'
        WHEN search_type ='feed' and source_view is Null and source is null then 'Лента ВК'
        ELSE 'Остальные'
    END
        
        
        
        
        """,con)

In [300]:
df_view_total.source.unique()

array(['Подписки', 'Антибарахолки', 'Антибарахолки портлеты', 'Поиск ВК',
       'Остальные', 'Лента ВКО', 'Карусели', 'Профиль продавца',
       'Портлеты', 'Неизвестно', 'Похожие товары', 'Группы',
       'Рекламные ссылки', 'Лента ВК', 'Пуш', 'Закладки', 'Сниппеты'],
      dtype=object)

In [301]:
df_view_total['cr_users']= round(df_view_total['c_users']/df_view_total['v_users']*100,2)
df_view_total['cr_events']= round(df_view_total['c_users']/df_view_total['v_users']*100,2)

In [302]:
df_view_total=df_view_total.sort_values('v.dt')

In [305]:
fig = px.line(df_view_total[(df_view_total['v.dt']>='2022-02-11')], x="v.dt", y="contact",color='source',title='Конверсия из просмотра в контакт')
fig.show()





In [282]:
df_contact_gr = df_view_total[df_view_total['v.dt']>='2022-02-11'].groupby(['v.dt', 'source']).agg({'contact': 'sum'})
totals = df_view_total[df_view_total['v.dt']>='2022-02-11'].groupby(['v.dt']).agg({'contact': 'sum'})
total_contact_perc = df_contact_gr.div(totals, level='v.dt') * 100
total_contact_perc = total_contact_perc.reset_index()

df_view_gr = df_view_total[df_view_total['v.dt']>='2022-02-11'].groupby(['v.dt', 'source']).agg({'views': 'sum'})
totals = df_view_total[df_view_total['v.dt']>='2022-02-11'].groupby(['v.dt']).agg({'views': 'sum'})
total_view_perc = df_view_gr.div(totals, level='v.dt') * 100
total_view_perc = total_view_perc.reset_index()



df_cr_gr = round(df_view_total[df_view_total['v.dt']>='2022-02-11'].groupby([ 'source']).agg({'cr_users': 'mean'}),2)



In [292]:
total_view_perc = round(total_view_perc.groupby('source').agg({'views': 'mean'}).sort_values(by='views',ascending=False),2)
total_contact_perc = round(total_contact_perc.groupby('source').agg({'contact': 'mean'}).sort_values(by='contact',ascending=False),2)

# cmap=sns.diverging_palette(5, 250, as_cmap=True)

total_view_perc.merge(total_contact_perc,how='left',on='source').merge(df_cr_gr,on='source').style.background_gradient(cmap='GnBu', axis=0)





,views,contact,cr_users
source,,,
Сниппеты,24.120000,12.930000,1.650000
Портлеты,16.170000,8.220000,1.810000
Лента ВКО,12.510000,19.440000,10.730000
Похожие товары,8.010000,6.370000,3.460000
Пуш,7.340000,5.630000,3.380000
Лента ВК,7.230000,1.920000,0.740000
Рекламные ссылки,5.030000,3.760000,2.890000
Профиль продавца,4.890000,13.200000,10.580000
Группы,4.180000,2.200000,1.950000


## Сниппеты

In [313]:
df_view_snippet = sql_to_df(""" 

select dt, 
    case when screen like 'feed%' then 'feed' else 'group' end as screen,
    --vk_platform,
    count(distinct event_id) cnt ,
    count(distinct user_id) users,
    count(distinct product_id) prods
from (
select coalesce( sender_id,client_user_id) user_id,
    event_id,
    product_id,
    CASE
        WHEN lower(client_agent) LIKE '%android%' THEN 'android'
        WHEN lower(client_agent) like '%iphone%' or lower(client_agent) like '%ios%' THEN 'ios'
        ELSE 'other'
    END AS vk_platform,
    search_type as screen,
    dt
from topics.y_user_product_counter_new m 
where dt>='2022-01-05' and dt<='2022-01-20'
  and event_name='prod.view'
  and coalesce(client_platform,platform)='vk_mini_app'
  and (search_type like 'group%' or search_type like 'feed%')
  and vk_source like 'wall%' 

union all

select  vk.user_id,
    concat(c.user_id,youla_item_id,time) event_id,
    youla_item_id as product_id,
    vk_platform,
    screen,
    c.dt
FROM topics.youla_vk_to_youla_pa2p_all_events c
join aggregations.cpa_vk_ids_by_dt vk on vk.dt=c.dt and vk.vk_id=c.user_id
WHERE c.dt>='2022-01-05' and c.dt<='2022-01-20'
    and event = 'transition_to_item'
    and content_type = 'wall'
    and (screen like 'group%' or screen like 'feed%')
    and vk_platform='web2'
) t 
group by dt
    , case when screen like 'feed%' then 'feed' else 'group' end
    --,vk_platform
    -- case when vk_platform like 'android' then 'android'
    --     when (vk_platform like 'ios' or vk_platform like 'iphone' or vk_platform like 'ipad') then 'ios'
    --     when (vk_platform like 'web' or vk_platform like 'web2') then 'web' 
    -- end 


""",con)

In [314]:
df_view_snippet=df_view_snippet.sort_values('dt')

fig = px.line(df_view_snippet, x="dt", y="cnt",color='screen',title='Просмотры из сниппетов (прил+веб)')
fig.show()

In [320]:
df_contact_snippet = sql_to_df(""" 
select dt,
    case when search_type like 'group%' then 'group' else 'feed' end search_type,
    count(distinct event_id) cnt,
    count(distinct user_id) users,
    count(distinct product_id) prods
from (
select c.*, search_type
from (select coalesce( sender_id,client_user_id) user_id,
    event_id,
    product_id,
    CASE
        WHEN lower(client_agent) LIKE '%android%' THEN 'android'
        WHEN lower(client_agent) like '%iphone%' THEN 'ios'
        ELSE 'other'
    END AS vk_platform,
    dt
from topics.y_user_user_counter
where dt>='2022-01-05' and dt<='2022-01-20'
  and event_name in ('usr.presscall','chat.first_message')
  and coalesce(client_platform,platform) in ('vk','vk_mini_app')
  and get_json_object(sources,'$.previous.screen')='snippet'
  ) c
join (
select user_id,
    product_id,
    dt,
    max(search_type) search_type
from (select coalesce( sender_id,client_user_id) user_id,
        FIRST_VALUE(search_type) over(partition by dt, coalesce(client_user_id,sender_id),product_id order by event_timestamp) search_type,
        product_id,
        dt
from topics.y_user_product_counter_new m 
where dt>='2022-01-05' and dt<='2022-01-20'
  and event_name='prod.view'
  and coalesce(client_platform,platform)='vk_mini_app'
  and (search_type like 'group%' or search_type like 'feed%')
  and vk_source like 'wall%'
      )v
group by user_id,
    product_id,
    dt
  ) v on v.user_id=c.user_id and v.dt=c.dt and c.product_id=v.product_id

union all

select vk.user_id,
    event_id,
    c.product_id,
    vk_platform,
    c.dt,
    search_type
from (select user_id,
    concat(user_id,youla_item_id,time) event_id,
    youla_item_id as product_id,
    vk_platform,
    dt
FROM topics.youla_vk_to_youla_pa2p_all_events
WHERE dt>='2022-01-05' and dt<='2022-01-20'
    and event in ('send_message','show_phone_click')
    and screen like 'group%'
    and vk_platform='web2') c
join aggregations.cpa_vk_ids_by_dt vk on vk.dt=c.dt and vk.vk_id=c.user_id
join (
select user_id, product_id,dt, max(search_type) search_type
from (select user_id,
    FIRST_VALUE(screen) over(partition by dt, user_id,youla_item_id order by time) search_type,   
    youla_item_id as product_id,
    dt
FROM topics.youla_vk_to_youla_pa2p_all_events
WHERE dt>='2022-01-05' and dt<='2022-01-20'
    and event = 'transition_to_item'
    and content_type = 'wall'
    and (screen like 'group%' or screen like 'feed%')
    and vk_platform='web2'
    )  v 
group by user_id, product_id,dt
    ) v on v.user_id=c.user_id and v.dt=c.dt and c.product_id=v.product_id ) t
group by dt
  , case when search_type like 'group%' then 'group' else 'feed' end

""",con)

In [321]:
df_contact_snippet=df_contact_snippet.sort_values('dt')

fig = px.line(df_contact_snippet, x="dt", y="cnt",color='search_type',title='Контакты из сниппетов (прил+веб)')
fig.show()

In [324]:
df_contact_snippet.columns=['dt', 'screen', 'cnt', 'users', 'prods']

In [326]:
df_snippet_xy = df_contact_snippet.merge(df_view_snippet[['dt','screen','users']],how='left',on=['dt','screen'])

In [328]:
df_snippet_xy['cr_contact']=round(df_snippet_xy['users_x']/df_snippet_xy['users_y']*100,2)

In [329]:
fig = px.line(df_snippet_xy, x="dt", y="cr_contact",color='screen',title='CR в контакт из сниппетов (прил+веб)')
fig.show()

In [332]:
df_show_snippet = sql_to_df(""" 
select dt,
    case when screen like 'group%' then 'group' else 'feed' end search_type,
    count(event) cnt,
    count(distinct user_id) users,
    count(distinct youla_item_id) prods
FROM topics.youla_vk_to_youla_pa2p_all_events
WHERE dt>='2022-01-05' and dt<='2022-01-20'
    and event = 'view'
    and content_type = 'wall'
    and (screen like 'group%' or screen like 'feed%')
group by dt
  , case when screen like 'group%' then 'group' else 'feed' end

""",con)

In [334]:
df_show_snippet=df_show_snippet.sort_values('dt')

fig = px.line(df_show_snippet, x="dt", y="cnt",color='search_type',title='Показы из сниппетов (прил+веб)')
fig.show()

In [335]:
df_show_snippet.columns=['dt', 'screen', 'cnt', 'users', 'prods']

In [336]:
df_snippet_xs = df_view_snippet.merge(df_show_snippet[['dt','screen','users']],how='left',on=['dt','screen'])

In [337]:
df_snippet_xs['cr_view']=round(df_snippet_xs['users_x']/df_snippet_xs['users_y']*100,2)

In [338]:

fig = px.line(df_snippet_xs, x="dt", y="cr_view",color='screen',title='СR в просмотр из сниппетов (прил+веб)')
fig.show()

In [147]:
df_view_total[(df_view_total.vk_source=='wall%') & (df_view_total.search_type.isin(['feed','group']))].source_view.unique()



array(['seller_profile', None, 'profile'], dtype=object)

In [131]:
dt_start='2022-01-10' 
dt_end ='2022-01-20'

In [166]:
df_contact_manual= sql_to_df("""


SELECT 'Сниппет группа' traffic_type,
                        cast(v_dt AS date),
                        count(DISTINCT v_event_id) cnt_view,
                        count(DISTINCT v_user_id) v_users,
                        count(DISTINCT v_product_id) v_prods,
                        count(DISTINCT c_event_id) cnt_con,
                        count(DISTINCT c_user_id) c_users,
                        count(DISTINCT c_product_id) c_prods
FROM
  (SELECT v.user_id AS v_user_id,
          v.dt AS v_dt,
          v.event_id AS v_event_id,
          v.product_id AS v_product_id,
          c.event_id c_event_id,
          c.user_id c_user_id,
          c.product_id AS c_product_id
   FROM
     (SELECT coalesce(sender_id,client_user_id) user_id,
             product_id,
             dt,
             count(DISTINCT event_id) event_id
      FROM topics.y_user_product_counter_new m
      WHERE dt>='{0}'
        AND dt<='{1}'
        AND event_name='prod.view'
        AND coalesce(client_platform,platform)='vk_mini_app'
        AND search_type = 'group'
        AND vk_source LIKE 'wall%'
      GROUP BY coalesce(sender_id,client_user_id),
               product_id,
               dt) v
   LEFT JOIN
     (SELECT coalesce(sender_id,client_user_id) user_id,
             event_id,
             product_id,
             CASE
                 WHEN lower(client_agent) LIKE '%android%' THEN 'android'
                 WHEN lower(client_agent) LIKE '%iphone%' THEN 'ios'
                 ELSE 'other'
             END AS vk_platform,
             dt
      FROM topics.y_user_user_counter
      WHERE dt>='{0}'
        AND dt<='{1}'
        AND event_name IN ('usr.presscall',
                           'chat.first_message')
        AND coalesce(client_platform,platform) = 'vk_mini_app'
        AND get_json_object(sources,'$.previous.screen') = 'snippet' ) c ON v.user_id=c.user_id
   AND v.dt=c.dt
   AND c.product_id=v.product_id)t
GROUP BY v_dt

  
UNION 

select 'Сниппет лента' traffic_type,
                        cast(v_dt AS date),
                        count(DISTINCT v_event_id) cnt_view,
                        count(DISTINCT v_user_id) v_users,
                        count(DISTINCT v_product_id) v_prods,
                        count(DISTINCT c_event_id) cnt_con,
                        count(DISTINCT c_user_id) c_users,
                        count(DISTINCT c_product_id) c_prods
FROM
  (SELECT v.user_id AS v_user_id,
          v.dt AS v_dt,
          v.event_id AS v_event_id,
          v.product_id AS v_product_id,
          c.event_id c_event_id,
          c.user_id c_user_id,
          c.product_id AS c_product_id
   FROM
     (SELECT coalesce(sender_id,client_user_id) user_id,
             product_id,
             dt,
             count(DISTINCT event_id) event_id
      FROM topics.y_user_product_counter_new m
      WHERE dt>='{0}'
        AND dt<='{1}'
        AND event_name='prod.view'
        AND coalesce(client_platform,platform)='vk_mini_app'
        AND search_type = 'feed'
        AND vk_source LIKE 'wall%'
      GROUP BY coalesce(sender_id,client_user_id),
               product_id,
               dt) v
   LEFT JOIN
     (SELECT coalesce(sender_id,client_user_id) user_id,
             event_id,
             product_id,
             CASE
                 WHEN lower(client_agent) LIKE '%android%' THEN 'android'
                 WHEN lower(client_agent) LIKE '%iphone%' THEN 'ios'
                 ELSE 'other'
             END AS vk_platform,
             dt
      FROM topics.y_user_user_counter
      WHERE dt>='{0}'
        AND dt<='{1}'
        AND event_name IN ('usr.presscall',
                           'chat.first_message')
        AND coalesce(client_platform,platform) = 'vk_mini_app'
        AND get_json_object(sources,'$.previous.screen') = 'snippet' ) c ON v.user_id=c.user_id
   AND v.dt=c.dt
   AND c.product_id=v.product_id)t
GROUP BY v_dt



UNION 


SELECT 'Портлеты' traffic_type,
                        cast(v_dt AS date),
                        count(DISTINCT v_event_id) cnt_view,
                        count(DISTINCT v_user_id) v_users,
                        count(DISTINCT v_product_id) v_prods,
                        count(DISTINCT c_event_id) cnt_con,
                        count(DISTINCT c_user_id) c_users,
                        count(DISTINCT c_product_id) c_prods
FROM
  (SELECT v.user_id AS v_user_id,
          v.dt AS v_dt,
          v.event_id AS v_event_id,
          v.product_id AS v_product_id,
          c.event_id c_event_id,
          c.user_id c_user_id,
          c.product_id AS c_product_id
   FROM
     (SELECT coalesce(sender_id,client_user_id) user_id,
             product_id,
             dt,
             count(DISTINCT event_id) event_id
      FROM topics.y_user_product_counter_new m
      WHERE dt>='{0}'
        AND dt<='{1}'
        AND event_name='prod.view'
        AND coalesce(client_platform,platform)='vk_mini_app'
        AND search_type = 'vk_recommendations'
      GROUP BY coalesce(sender_id,client_user_id),
               product_id,
               dt) v
   LEFT JOIN
     (SELECT coalesce(sender_id,client_user_id) user_id,
             event_id,
             product_id,
             CASE
                 WHEN lower(client_agent) LIKE '%android%' THEN 'android'
                 WHEN lower(client_agent) LIKE '%iphone%' THEN 'ios'
                 ELSE 'other'
             END AS vk_platform,
             dt
      FROM topics.y_user_user_counter
      WHERE dt>='{0}'
        AND dt<='{1}'
        AND event_name IN ('usr.presscall',
                           'chat.first_message')
        AND coalesce(client_platform,platform) = 'vk_mini_app'
        AND get_json_object(sources,'$.previous.screen') = 'vk_recommendations' ) c ON v.user_id=c.user_id
   AND v.dt=c.dt
   AND c.product_id=v.product_id)t
GROUP BY v_dt




UNION 

select 'Главная ВКО'  traffic_type,
                        cast(v_dt AS date),
                        count(DISTINCT v_event_id) cnt_view,
                        count(DISTINCT v_user_id) v_users,
                        count(DISTINCT v_product_id) v_prods,
                        count(DISTINCT c_event_id) cnt_con,
                        count(DISTINCT c_user_id) c_users,
                        count(DISTINCT c_product_id) c_prods
FROM
  (SELECT v.user_id AS v_user_id,
          v.dt AS v_dt,
          v.event_id AS v_event_id,
          v.product_id AS v_product_id,
          c.event_id c_event_id,
          c.user_id c_user_id,
          c.product_id AS c_product_id
   FROM
     (SELECT coalesce(sender_id,client_user_id) user_id,
             product_id,
             dt,
             search_id,
             count(DISTINCT event_id) event_id
      FROM topics.y_user_product_counter_new m
      WHERE dt>='{0}'
        AND dt<='{1}'
        AND event_name='prod.view'
        AND coalesce(client_platform,platform)='vk_mini_app'
      GROUP BY coalesce(sender_id,client_user_id),
               product_id,
               search_id,
               dt) v
JOIN
    (SELECT search_id
    FROM topics.y_searches
    WHERE coalesce(client_platform,platform)='vk_mini_app'
        AND dt>='{0}'
        AND dt<='{1}'
        and size(categories)>1) srch on srch.search_id=v.search_id
LEFT JOIN
     (SELECT coalesce(sender_id,client_user_id) user_id,
             event_id,
             product_id,
             CASE
                 WHEN lower(client_agent) LIKE '%android%' THEN 'android'
                 WHEN lower(client_agent) LIKE '%iphone%' THEN 'ios'
                 ELSE 'other'
             END AS vk_platform,
             dt
      FROM topics.y_user_user_counter
      WHERE dt>='{0}'
        AND dt<='{1}'
        AND event_name IN ('usr.presscall',
                           'chat.first_message')
        AND coalesce(client_platform,platform) = 'vk_mini_app') c ON v.user_id=c.user_id
   AND v.dt=c.dt
   AND c.product_id=v.product_id)t
GROUP BY v_dt


""".format(dt_start,dt_end),con)

In [167]:
df_contact_manual

,_u4.traffic_type,_u4._c1,_u4.cnt_view,_u4.v_users,_u4.v_prods,_u4.cnt_con,_u4.c_users,_u4.c_prods
0,Сниппет лента,2022-01-12,20,488098,119854,3438,3211,2926
1,Сниппет группа,2022-01-12,15,142171,107222,4559,4028,4091
2,Сниппет лента,2022-01-14,13,419393,106271,2867,2706,2481
3,Главная ВКО,2022-01-13,11,45905,129850,3451,2318,3029
4,Главная ВКО,2022-01-19,15,43927,125412,3420,2228,3012
5,Главная ВКО,2022-01-16,22,49336,144584,3841,2453,3361
6,Портлеты,2022-01-18,19,248603,89855,4677,4183,3176
7,Главная ВКО,2022-01-15,14,45907,133213,3566,2326,3146
8,Портлеты,2022-01-17,15,259450,92138,4658,4190,3159
9,Портлеты,2022-01-13,15,281245,120177,5344,4807,3654


In [135]:
df_contact_manual= sql_to_df("""


select 'Сниппет группа' traffic_type,
    cast(v_dt as date),
    count(distinct v_event_id) cnt_view,
    count(distinct v_user_id) v_users,
    count(distinct v_product_id) v_prods,
    count(distinct c_event_id) cnt_con,
    count(distinct c_user_id) c_users,
    count(distinct c_product_id) c_prods
from (
select v.user_id as v_user_id,
    v.dt as v_dt,
    v.event_id as v_event_id,
    v.product_id as v_product_id,
    c.event_id c_event_id,
    c.user_id s c_user_id,
    c.product_id as c_product_id
from  (select distinct coalesce( sender_id,client_user_id) user_id,
        product_id,
        dt
from topics.y_user_product_counter_new m 
where dt>='{dt_start}' and dt<='{dt_end}'
  and event_name='prod.view'
  and coalesce(client_platform,platform)='vk_mini_app'
  and search_type = 'group'
  and vk_source like 'wall%'
  ) v 
left join 
(select coalesce( sender_id,client_user_id) user_id,
    event_id,
    product_id,
    CASE
        WHEN lower(client_agent) LIKE '%android%' THEN 'android'
        WHEN lower(client_agent) like '%iphone%' THEN 'ios'
        ELSE 'other'
    END AS vk_platform,
    dt
from topics.y_user_user_counter
where dt>='{dt_start}' and dt<='{dt_end}'
  and event_name in ('usr.presscall','chat.first_message')
  and coalesce(client_platform,platform) = 'vk_mini_app'
  and get_json_object(sources,'$.previous.screen') = 'snippet'
  ) c on v.user_id=c.user_id and v.dt=c.dt and c.product_id=v.product_id


union all


select vk.user_id as v_user_id,
    v.dt as v_dt,
    v.event_id as v_event_id,
    v.product_id as v_product_id,
    c.event_id c_event_id,
    c.user_id s c_user_id,
    c.product_id as c_product_id
from (select distinct user_id,
    youla_item_id as product_id,
    dt
FROM topics.youla_vk_to_youla_pa2p_all_events
WHERE dt>='{dt_start}' and dt<='{dt_end}'
    and event = 'transition_to_item'
    and content_type = 'wall'
    and screen like 'group%'
    and vk_platform in ('web2')
    ) v 
join aggregations.cpa_vk_ids_by_dt vk on vk.dt=v.dt and vk.vk_id=v.user_id
left join (select user_id,
    concat(user_id,youla_item_id,time) event_id,
    youla_item_id as product_id,
    vk_platform,
    dt
FROM topics.youla_vk_to_youla_pa2p_all_events
WHERE dt>='{dt_start}' and dt<='{dt_end}'
    and event in ('send_message','show_phone_click')
    and screen like 'group%'
    and vk_platform = 'web2') c on v.user_id=c.user_id and v.dt=c.dt and c.product_id=v.product_id ) t
group by cast(dt as date)
 
  
UNION 

select 'Сниппет лента' traffic_type,
    cast(dt as date),
    count(distinct event_id) cnt,
    count(distinct user_id) users,
    count(distinct product_id) prods
from (
select c.*
from (select coalesce( sender_id,client_user_id) user_id,
    event_id,
    product_id,
    CASE
        WHEN lower(client_agent) LIKE '%android%' THEN 'android'
        WHEN lower(client_agent) like '%iphone%' THEN 'ios'
        ELSE 'other'
    END AS vk_platform,
    dt
from topics.y_user_user_counter
where dt>='{dt_start}' and dt<='{dt_end}'
  and event_name in ('usr.presscall','chat.first_message')
  and coalesce(client_platform,platform) in ('vk','vk_mini_app')
  and get_json_object(sources,'$.previous.screen') = 'snippet'
  ) c
join (select distinct coalesce( sender_id,client_user_id) user_id,
        product_id,
        dt
from topics.y_user_product_counter_new m 
where dt>='{dt_start}' and dt<='{dt_end}'
  and event_name='prod.view'
  and coalesce(client_platform,platform)='vk_mini_app'
  --and search_type = 'vk_recommendations'
  and search_type = 'feed'
  and vk_source like 'wall%'
  ) v on v.user_id=c.user_id and v.dt=c.dt and c.product_id=v.product_id

union all

select vk.user_id,
    event_id,
    c.product_id,
    vk_platform,
    c.dt
from (select user_id,
    concat(user_id,youla_item_id,time) event_id,
    youla_item_id as product_id,
    vk_platform,
    dt
FROM topics.youla_vk_to_youla_pa2p_all_events
WHERE dt>='{dt_start}' and dt<='{dt_end}'
    and event in ('send_message','show_phone_click')
    and screen like 'feed%'
    and vk_platform='web2') c
join aggregations.cpa_vk_ids_by_dt vk on vk.dt=c.dt and vk.vk_id=c.user_id
join (select distinct user_id,
    youla_item_id as product_id,
    dt
FROM topics.youla_vk_to_youla_pa2p_all_events
WHERE dt>='{dt_start}' and dt<='{dt_end}'
    and event = 'transition_to_item'
    --and collection_type = 'recommendations'
    and content_type = 'wall'
    and screen like 'feed%'
    and vk_platform='web2'
    ) v on v.user_id=c.user_id and v.dt=c.dt and c.product_id=v.product_id ) t
group by cast(dt as date)


  
UNION 

select 'Портлеты' traffic_type,
    cast(dt as date),
    count(distinct event_id) cnt,
    count(distinct user_id) users,
    count(distinct product_id) prods
from (
select coalesce( sender_id,client_user_id) user_id,
    event_id,
    product_id,
    CASE
        WHEN lower(client_agent) LIKE '%android%' THEN 'android'
        WHEN lower(client_agent) like '%iphone%' THEN 'ios'
        ELSE 'other'
    END AS vk_platform,
    dt
from topics.y_user_user_counter
where dt>='{dt_start}' and dt<='{dt_end}'
  and event_name in ('usr.presscall','chat.first_message')
  and coalesce(client_platform,platform) in ('vk','vk_mini_app')
  and get_json_object(sources,'$.previous.screen') = 'vk_recommendations'

union all

select vk.user_id,
    event_id,
    c.product_id,
    vk_platform,
    c.dt
from (select user_id,
    concat(user_id,youla_item_id,time) event_id,
    youla_item_id as product_id,
    vk_platform,
    dt
  FROM topics.youla_vk_to_youla_pa2p_all_events
  WHERE dt>='{dt_start}' and dt<='{dt_end}'
    and event in ('send_message','show_phone_click')
    and screen like 'feed%'
    and vk_platform='web2') c
join aggregations.cpa_vk_ids_by_dt vk on vk.dt=c.dt and vk.vk_id=c.user_id
join (select distinct user_id,
      youla_item_id as product_id,
      dt
  FROM topics.youla_vk_to_youla_pa2p_all_events
  WHERE dt>='{dt_start}' and dt<='{dt_end}'
    and event = 'transition_to_item'
    and collection_type = 'recommendations'
    -- and content_type = 'wall'
    -- and screen like 'feed%'
    and vk_platform='web2'
    ) v on v.user_id=c.user_id and v.dt=c.dt and c.product_id=v.product_id 
  ) t
group by cast(dt as date)



UNION 

select 'Главная ВКО' traffic_type,
    cast(dt as date),
    count(distinct event_id) cnt,
    count(distinct user_id) users,
    count(distinct product_id) prods
from (
select c.user_id,
    event_id,
    c.product_id,
    vk_platform,
    c.dt
from (
select coalesce( sender_id,client_user_id) user_id,
    event_id,
    product_id,
    CASE
        WHEN lower(client_agent) LIKE '%android%' THEN 'android'
        WHEN lower(client_agent) like '%iphone%' THEN 'ios'
        ELSE 'other'
    END AS vk_platform,
    search_id,
    dt
from topics.y_user_user_counter
where dt>='{dt_start}' and dt<='{dt_end}'
  and event_name in ('usr.presscall','chat.first_message')
  and coalesce(client_platform,platform) in ('vk','vk_mini_app')
  ) c
join (select search_id
    from topics.y_searches
    where coalesce(client_platform,platform)='vk_mini_app'
        and dt>='{dt_start}' and dt<='{dt_end}'
        and size(categories)>1) srch on srch.search_id=c.search_id


union all

select vk.user_id,
    event_id,
    c.product_id,
    vk_platform,
    c.dt
from (select user_id,
    concat(user_id,youla_item_id,time) event_id,
    youla_item_id as product_id,
    vk_platform,
    dt
FROM topics.youla_vk_to_youla_pa2p_all_events
WHERE dt>='{dt_start}' and dt<='{dt_end}'
    and event in ('send_message','show_phone_click')
    and screen = 'vko'
    and vk_platform='web2') c
join aggregations.cpa_vk_ids_by_dt vk on vk.dt=c.dt and vk.vk_id=c.user_id
) t
group by cast(dt as date)




UNION 

select 'Антибарахолки' traffic_type,
    cast(dt as date),
    count(distinct event_id) cnt,
    count(distinct user_id) users,
    count(distinct product_id) prods
from (

select vk.user_id,
    event_id,
    c.product_id,
    vk_platform,
    c.dt
from (select user_id,
    concat(user_id,youla_item_id,time) event_id,
    youla_item_id as product_id,
    vk_platform,
    dt
FROM topics.youla_vk_to_youla_pa2p_all_events
WHERE dt>='{dt_start}' and dt<='{dt_end}'
   and
        ((event = 'send_message' and screen in ('group', 'classifieds_products', 'classified') and 
            (community_classified_type = 'anticlassifieds'or classified_setting = 2))  
       or ( event = 'show_phone_click' and screen in ('group', 'classifieds_products', 'classified', 'other') and 
            (community_classified_type = 'anticlassifieds' or classified_setting = 2))
       or (event = 'miniapp_show_phone_click' and youla_ref in ('lent_from_widget', 'widget'))
       or (event = 'miniapp_send_message_click' and youla_ref in ('lent_from_widget', 'widget'))
      or  (event in ('miniapp_send_message_click', 'miniapp_show_phone_click') and youla_ref = 'widget_from_post')
      or  (event in ('send_message', 'show_phone_click') and content_type = 'wall_post' and classified_setting = 2
          ))) c
join aggregations.cpa_vk_ids_by_dt vk on vk.dt=c.dt and vk.vk_id=c.user_id
) t
group by cast(dt as date)


UNION 

select 'Пуши' traffic_type,
    cast(dt as date),
    count(distinct event_id) cnt,
    count(distinct user_id) users,
    count(distinct product_id) prods
from (
select coalesce( sender_id,client_user_id) user_id,
    event_id,
    product_id,
    CASE
        WHEN lower(client_agent) LIKE '%android%' THEN 'android'
        WHEN lower(client_agent) like '%iphone%' THEN 'ios'
        ELSE 'other'
    END AS vk_platform,
    dt
from topics.y_user_user_counter
where dt>='{dt_start}' and dt<='{dt_end}'
  and event_name in ('usr.presscall','chat.first_message')
  and coalesce(client_platform,platform) in ('vk','vk_mini_app')
  and get_json_object(sources,'$.previous.screen') = 'push'
  ) c
group by cast(dt as date)  
  
UNION 

select 'Поиск в ВКО'traffic_type,
    cast(dt as date),
    count(distinct event_id) cnt,
    count(distinct user_id) users,
    count(distinct product_id) prods
from (
select coalesce( sender_id,client_user_id) user_id,
    event_id,
    product_id,
    CASE
        WHEN lower(client_agent) LIKE '%android%' THEN 'android'
        WHEN lower(client_agent) like '%iphone%' THEN 'ios'
        ELSE 'other'
    END AS vk_platform,
    search_id,
    dt
from topics.y_user_user_counter
where dt>='{dt_start}' and dt<='{dt_end}'
  and event_name in ('usr.presscall','chat.first_message')
  and coalesce(client_platform,platform) in ('vk','vk_mini_app')
  and coalesce(get_json_object(sources,'$.previous.screen'),'other') <> 'push'
 
  ) c
join (select search_id
    from topics.y_searches
    where coalesce(client_platform,platform)='vk_mini_app'
        and dt>='2022-01-10' and dt<='2022-01-23'
        and (search_text is not null or category is not null or subcategory is not null)
        ) srch on srch.search_id=c.search_id
group by cast(dt as date)


UNION 

select 'Подписки' traffic_type,
    cast(dt as date),
    count(distinct event_id) cnt,
    count(distinct user_id) users,
    count(distinct product_id) prods
from (
select coalesce( sender_id,client_user_id) user_id,
    event_id,
    product_id,
    CASE
        WHEN lower(client_agent) LIKE '%android%' THEN 'android'
        WHEN lower(client_agent) like '%iphone%' THEN 'ios'
        ELSE 'other'
    END AS vk_platform,
    dt
from topics.y_user_user_counter
where dt>='{dt_start}' and dt<='{dt_end}'
  and event_name in ('usr.presscall','chat.first_message')
  and coalesce(client_platform,platform) in ('vk','vk_mini_app')
  and get_json_object(sources,'$.previous.screen') in  ('subscribe_similar','seller_subscribe','personal_search')
  ) c
group by cast(dt as date)

UNION 

select 'Похожие' traffic_type,
    cast(dt as date),
    count(distinct event_id) cnt,
    count(distinct user_id) users,
    count(distinct product_id) prods
from (
select coalesce( sender_id,client_user_id) user_id,
    event_id,
    product_id,
    CASE
        WHEN lower(client_agent) LIKE '%android%' THEN 'android'
        WHEN lower(client_agent) like '%iphone%' THEN 'ios'
        ELSE 'other'
    END AS vk_platform,
    dt
from topics.y_user_user_counter
where dt>='{dt_start}' and dt<='{dt_end}'
  and event_name in ('usr.presscall','chat.first_message')
  and coalesce(client_platform,platform) in ('vk','vk_mini_app')
  and get_json_object(sources,'$.previous.screen') = 'similars'
  ) c
group by cast(dt as date)


-- UNION 

-- select 'Рекламные посты в ленте'traffic_type,
--     cast(dt as date),
--     count(distinct event_id) cnt,
--     count(distinct user_id) users,
--     count(distinct product_id) prods
-- from (
-- select coalesce( sender_id,client_user_id) user_id,
--     event_id,
--     product_id,
--     CASE
--         WHEN lower(client_agent) LIKE '%android%' THEN 'android'
--         WHEN lower(client_agent) like '%iphone%' THEN 'ios'
--         ELSE 'other'
--     END AS vk_platform,
--     dt
-- from topics.y_user_user_counter
-- where dt>='{dt_start}' and dt<='{dt_end}'
--   and event_name in ('usr.presscall','chat.first_message')
--   and coalesce(client_platform,platform) in ('vk','vk_mini_app')
--   and get_json_object(sources,'$.previous.screen') = 'vk_lent_advertisement'
--   ) c
-- group by cast(dt as date)
""",con)

OperationalError: TExecuteStatementResp(status=TStatus(statusCode=3, infoMessages=['*org.apache.hive.service.cli.HiveSQLException:Error while processing statement: FAILED: Execution Error, return code 1 from org.apache.hadoop.hive.ql.exec.tez.TezTask. Application killed by user.:28:27', 'org.apache.hive.service.cli.operation.Operation:toSQLException:Operation.java:324', 'org.apache.hive.service.cli.operation.SQLOperation:runQuery:SQLOperation.java:199', 'org.apache.hive.service.cli.operation.SQLOperation:runInternal:SQLOperation.java:230', 'org.apache.hive.service.cli.operation.Operation:run:Operation.java:264', 'org.apache.hive.service.cli.session.HiveSessionImpl:executeStatementInternal:HiveSessionImpl.java:479', 'org.apache.hive.service.cli.session.HiveSessionImpl:executeStatement:HiveSessionImpl.java:454', 'sun.reflect.GeneratedMethodAccessor93:invoke::-1', 'sun.reflect.DelegatingMethodAccessorImpl:invoke:DelegatingMethodAccessorImpl.java:43', 'java.lang.reflect.Method:invoke:Metho